### Complex Schema Validation with Avro
**Description**: Implement a solution in Python to validate records against a complex nested Avro schema.

Eg., Complex schema ( nested_schema.avsc ):

**Steps**:
1. Load schema
2. Example data to validate
3. Validate against schema
4. Read back to check

In [1]:
# write your code from here
import json
from avro.schema import make_avsc_object, SchemaParseException
from avro.io import DatumWriter, DatumReader, BinaryEncoder, BinaryDecoder
import io

def validate_avro_record(schema_file, records_to_validate):
    """
    Validates records against a complex nested Avro schema.

    Args:
        schema_file (str): Path to the Avro schema file (.avsc or .json).
        records_to_validate (list): A list of Python dictionaries representing
                                     the records to validate.
    """
    try:
        # Step 1: Load schema
        with open(schema_file, 'r') as f:
            schema_json = json.load(f)
        avro_schema = make_avsc_object(schema_json)
        print(f"Avro schema loaded successfully from: {schema_file}\n")
        print("Avro Schema:")
        print(avro_schema.to_json(indent=2))

        # Step 3: Validate against schema
        writer = DatumWriter(avro_schema)
        reader = DatumReader(avro_schema)
        validation_results = []

        print("\nValidation Results:")
        for i, record in enumerate(records_to_validate):
            try:
                # Encode the record (this implicitly validates against the schema)
                buffer = io.BytesIO()
                encoder = BinaryEncoder(buffer)
                writer.write(record, encoder)
                encoded_record = buffer.getvalue()

                # Step 4: Read back to check (optional, but confirms encoding/decoding)
                buffer.seek(0)
                decoder = BinaryDecoder(buffer)
                decoded_record = reader.read(decoder)

                validation_results.append((f"Record {i+1}", "Valid"))
                print(f"Record {i+1}: Valid")

            except SchemaParseException as e:
                validation_results.append((f"Record {i+1}", f"Invalid: {e}"))
                print(f"Record {i+1}: Invalid - Schema Mismatch: {e}")
            except Exception as e:
                validation_results.append((f"Record {i+1}", f"Invalid: {e}"))
                print(f"Record {i+1}: Invalid - Other Error: {e}")

        return validation_results

    except FileNotFoundError:
        print(f"Error: Schema file not found at {schema_file}")
        return None
    except json.JSONDecodeError:
        print(f"Error: Invalid JSON format in schema file: {schema_file}")
        return None
    except SchemaParseException as e:
        print(f"Error: Could not parse Avro schema: {e}")
        return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Step 2: Example data to validate
# Assuming the following nested schema (nested_schema.avsc):
"""
{
  "type": "record",
  "name": "User",
  "fields": [
    {"name": "name", "type": "string"},
    {
      "name": "address",
      "type": {
        "type": "record",
        "name": "Address",
        "fields": [
          {"name": "street", "type": "string"},
          {"name": "city", "type": "string"},
          {"name": "zip", "type": "int"}
        ]
      }
    },
    {"name": "emails", "type": {"type": "array", "items": "string"}},
    {
      "name": "preferences",
      "type": ["null", {"type": "map", "values": "string"}]
    }
  ]
}
"""

# Create a sample schema file
schema_content = {
  "type": "record",
  "name": "User",
  "fields": [
    {"name": "name", "type": "string"},
    {
      "name": "address",
      "type": {
        "type": "record",
        "name": "Address",
        "fields": [
          {"name": "street", "type": "string"},
          {"name": "city", "type": "string"},
          {"name": "zip", "type": "int"}
        ]
      }
    },
    {"name": "emails", "type": {"type": "array", "items": "string"}},
    {
      "name": "preferences",
      "type": ["null", {"type": "map", "values": "string"}]
    }
  ]
}
schema_file = 'nested_schema.avsc'
with open(schema_file, 'w') as f:
    json.dump(schema_content, f, indent=2)
print(f"\nSample Avro schema created at: {schema_file}")

records_to_validate = [
    {"name": "Alice", "address": {"street": "123 Main St", "city": "Bengaluru", "zip": 560001}, "emails": ["alice@example.com"], "preferences": {"color": "blue"}},
    {"name": "Bob", "address": {"street": "456 Park Ave", "city": "Mumbai", "zip": "400001"}, "emails": ["bob@example.com", "bob.work@example.com"], "preferences": None},
    {"name": "Charlie", "address": {"street": "789 Pine Ln", "city": "Delhi", "zip": 110001}, "emails": "charlie@example.com", "preferences": {1: "one"}}, # Incorrect email format, incorrect preferences key type
    {"name": "Diana", "address": {"street": "10 Downing St"}, "emails": [], "preferences": {}}, # Missing city and zip
    {"name": 123, "address": {"street": 234, "city": 567, "zip": "invalid"}, "emails": [1, 2], "preferences": {"active": True}} # Incorrect data types
]

# Validate the records
validation_results = validate_avro_record(schema_file, records_to_validate)

if validation_results:
    print("\nDetailed Validation Report:")
    for record_id, result in validation_results:
        print(f"{record_id}: {result}")

ModuleNotFoundError: No module named 'avro'